<h1><font color='red'>NOTE</font> - All the API keys removed from the code, as the keys are not advised to be shared.</h1>

<h3>Here, I am using SerpAPI (https://serpapi.com/) to scrape the required data. It's a paid platform with a free tier option as well, where you can send 100 requests in a month.</h3>

**NOTE -** I tried using selenium, was able to scrape some of the data but google does not allow to scrape everything. So opted out from the selenium approach. Then tried Google Maps API using free tier. It allows only 5 reviews to scrape from the maps. So opted from that as well.

<h1>1. Scraping Google Maps (perpustakaan ui) reviews</h1>

In [1]:
# required installation for serpapi

!pip install google-search-results

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32019 sha256=b86fb49b85b0b55f356cda0a97b120abe63303509da9a577f38ea9849d328359
  Stored in directory: /root/.cache/pip/wheels/d3/b2/c3/03302d12bb44a2cdff3c9371f31b72c0c4e84b8d2285eeac53
Successfully built google-search-results


<h3>Passing parameters to API, getting the scraped data, extending the scraping of data till all the reviews</h3>

In [1]:
#imports
from serpapi import GoogleSearch
import pandas as pd
from urllib.parse import urlsplit, parse_qsl
import json

In [9]:
# parameters passed for API call (scraping all the map data of that place)
# api_key - it is the unique key which we get from serpapi
# engine - SerpApi search engine (google maps, google maps reviews...there are multiple options to choose)
# q - query (here, our map location/place)
# ll - longitude & latitude (GPS coordinates)
# type - here we expect place result. so place option choosed.
# place_id - unique ID of the place (you can get place_id by using google maps API)
# hl - language choosen (here id is Indonesian)
# start - useful for pagination in upcoming steps (to gather all the reviews)
params = {
  'api_key': '<serp api key>',
  'engine': 'google_maps',
  'q': 'Universitas Indonesia Library',
  'll': '@-6.3652214,106.8269946,17z',
  'type': 'place',
  'place_id':'ChIJvYVjmBDsaS4Rr9Bc9B5a1VU',
  'hl': 'id',
  'start': 0,  
}

# make API call, fetch results into dictionary format
search = GoogleSearch(params)
results = search.get_dict()

# parameters passed for API call (scraping only the data related to reviews)
# engine - engine here will be google_maps_reviews
# sort_by - basically sorting most relevant reviews
# data_id - ID of the data (can be obtained through google maps API call or from the place URL.)
params = {
        'api_key': '143f3b60ab66b0347d1c589cfcfcc9a2ecd30c2f7db02563aa3b322737825c7a',
        'engine': 'google_maps_reviews',
        'sort_by': 'qualityScore',
        'hl': 'id',
        'data_id': '0x2e69ec10986385bd:0x55d55a1ef45cd0af'
    }

# make API call for reviews related data
search = GoogleSearch(params)

In [ ]:
# code snippet for pagination (getting all the reviews)

# empty list to store reviews data
reviews = []

# there is a condition inside the loop. If the next_page_token is present, then the search object is updated. Else, the loop is stopped
while True:
    new_reviews_page_result = search.get_dict()
    reviews.extend(new_reviews_page_result['reviews'])
    
    if new_reviews_page_result.get('serpapi_pagination').get('next') and new_reviews_page_result.get('serpapi_pagination').get('next_page_token'):
        search.params_dict.update(dict(parse_qsl(urlsplit(new_reviews_page_result.get('serpapi_pagination', {}).get('next')).query)))
    else:
        break

<h3>Extracting the reviewers names and their reviews, modifying the dataframe to have only Indonesian reviews</h3>

In [ ]:
# empty dict to store names and reviews
reviews_dict={}

# empty lists to store names and reviews separately
name_lst, review_lst=[],[]

for i in reviews:
  # storing names and reviews in the respective lists
  name_lst.append(i['user']['name'])
  review_lst.append(i['snippet'])

# storing the names and reviews list into the dictionary, so that it can be converted into a dataframe
reviews_dict['name'] = name_lst
reviews_dict['review'] = review_lst

# create a dataframe
df = pd.DataFrame(reviews_dict)
print(df)

                  name                                             review
0         Dwi Oktarina  Perpustakaan UI bebas diakses sivitas akademik...
1             Ramdanih  Lokasi yang strategis di kawasan Universitas I...
2    Putri Nurfitriani  Nyaman untuk ngerjain tugas buat yg males di k...
3         Kevin Ewaldo  Perpustakaan UI sudah sangat baik. Kalau bicar...
4             Emad Eus  Perpustakaan yg baik, namun maintenance infras...
..                 ...                                                ...
908      Jarot Manurat                                                   
909      Intan Hanifah                                                   
910   Estu widi Astari                                                   
911          User Name                                                   
912   Andri faisol dwi                                                   

[913 rows x 2 columns]


**Observation -** If observed carefully, the API extracted the original Indonesian reviews alongwith the English reviews (here the English reviews has its Indonesian translation as well). We don't need the actual English reviews. <br>
As we sorted the data before, we can locate where the Indonesian reviews finishes. It finishes at 310th index. So we'll just consider rows till 310 index(including 310th index).

In [ ]:
# first 311 rows
modified_df = df[:311]
print(modified_df)

                                            name  \
0                                   Dwi Oktarina   
1                                       Ramdanih   
2                              Putri Nurfitriani   
3                                   Kevin Ewaldo   
4                                       Emad Eus   
..                                           ...   
306                                     Makayasa   
307                           warnawarni florist   
308                                           3U   
309  Fajariah Tri Lestari (TheFamousz Gorgeousz)   
310                               Viester Dolles   

                                                review  
0    Perpustakaan UI bebas diakses sivitas akademik...  
1    Lokasi yang strategis di kawasan Universitas I...  
2    Nyaman untuk ngerjain tugas buat yg males di k...  
3    Perpustakaan UI sudah sangat baik. Kalau bicar...  
4    Perpustakaan yg baik, namun maintenance infras...  
..                               

<h3>Generating output csv file for indonesian reviews (perpustakaan ui)</h3>

In [ ]:
modified_df.to_csv('google_maps_indo_reviews.csv')
print('CSV generated!!')

CSV generated!!


<h1>2. Scraping Google Maps (perpustakaan ugm) reviews</h1>

In [13]:
# parameters passed for API call (scraping all the map data of that place)
# api_key - it is the unique key which we get from serpapi
# engine - SerpApi search engine (google maps, google maps reviews...there are multiple options to choose)
# q - query (here, our map location/place)
# ll - longitude & latitude (GPS coordinates)
# type - here we expect place result. so place option choosed.
# place_id - unique ID of the place (you can get place_id by using google maps API)
# hl - language choosen (here id is Indonesian)
# start - useful for pagination in upcoming steps (to gather all the reviews)
params = {
  'api_key': '<serp api key>',
  'engine': 'google_maps',
  'q': 'Gadjah Mada University Library perpustakaan ugm',
  'll': '@-7.769715,110.3754651,16z',
  'type': 'place',
  'place_id':'ChIJpY1OrExYei4RRukgnZ1MxpE',
  'hl': 'id',
  'start': 0,  
}

# make API call, fetch results into dictionary format
search = GoogleSearch(params)
results = search.get_dict()

# parameters passed for API call (scraping only the data related to reviews)
# engine - engine here will be google_maps_reviews
# sort_by - basically sorting most relevant reviews
# data_id - ID of the data (can be obtained through google maps API call or from the place URL.)
params = {
        'api_key': '143f3b60ab66b0347d1c589cfcfcc9a2ecd30c2f7db02563aa3b322737825c7a',
        'engine': 'google_maps_reviews',
        'sort_by': 'qualityScore',
        'hl': 'id',
        'data_id': '0x2e7a584cac4e8da5:0x91c64c9d9d20e946'
    }

# make API call for reviews related data
search = GoogleSearch(params)

In [15]:
# code snippet for pagination (getting all the reviews)

# empty list to store reviews data
reviews = []

# there is a condition inside the loop. If the next_page_token is present, then the search object is updated. Else, the loop is stopped
while True:
    new_reviews_page_result = search.get_dict()
    reviews.extend(new_reviews_page_result['reviews'])
    
    if new_reviews_page_result.get('serpapi_pagination').get('next') and new_reviews_page_result.get('serpapi_pagination').get('next_page_token'):
        search.params_dict.update(dict(parse_qsl(urlsplit(new_reviews_page_result.get('serpapi_pagination', {}).get('next')).query)))
    else:
        break

<h3>Extracting the reviewers names and their reviews, modifying the dataframe to have only Indonesian reviews</h3>

In [17]:
# empty dict to store names and reviews
reviews_dict={}

# empty lists to store names and reviews separately
name_lst, review_lst=[],[]

for i in reviews:
  # storing names and reviews in the respective lists
  name_lst.append(i['user']['name'])
  review_lst.append(i['snippet'])

# storing the names and reviews list into the dictionary, so that it can be converted into a dataframe
reviews_dict['name'] = name_lst
reviews_dict['review'] = review_lst

# create a dataframe
df = pd.DataFrame(reviews_dict)
df

,name,review
0,Vincentia Litta,"Untuk mahasiswa UGM, kalau mau dateng reservas..."
1,Djojos Soer,"Tempatnya bersih, tenang, dan teratur. Ruangan..."
2,Fajar Arifin,Salah satu tempat favorit di Jogja. Sangat nya...
3,Adinda Annisa Diva Pratiwi,Untuk ruang sirkulasi tempatnya nyaman. Koleks...
4,Mutia Z,Harus pesan atau booking kursi dulu sebelum ma...
...,...,...
738,Petrus Andhika,
739,18_Azzura Salsabila,
740,sls,
741,21_Laras Tristanti,


**Observation -** If observed carefully, the API extracted the original Indonesian reviews alongwith the English reviews (here the English reviews has its Indonesian translation as well). We don't need the actual English reviews. <br>
As we sorted the data before, we can locate where the Indonesian reviews finishes. It finishes at 240th index. So we'll just consider rows till 240 index (including 240th index).

In [18]:
# first 241 rows
modified_df = df[:241]
modified_df

,name,review
0,Vincentia Litta,"Untuk mahasiswa UGM, kalau mau dateng reservas..."
1,Djojos Soer,"Tempatnya bersih, tenang, dan teratur. Ruangan..."
2,Fajar Arifin,Salah satu tempat favorit di Jogja. Sangat nya...
3,Adinda Annisa Diva Pratiwi,Untuk ruang sirkulasi tempatnya nyaman. Koleks...
4,Mutia Z,Harus pesan atau booking kursi dulu sebelum ma...
...,...,...
236,Abiyadi Hartawanto,Kampus anak
237,Hafid Rusidi,Perpustakaan pusat ugm
238,Kewannian,Perpustakaanku 🔥🔥🔥
239,Syifa Nuril K.,"Perpusat! Selain karena deket sama FISIPOL, te..."


<h3>Generating output csv file for indonesian reviews (perpustakaan ugm)</h3>

In [19]:
modified_df.to_csv('google_maps_indo_reviews_perpustakaan_ugm.csv')
print('CSV generated!!')

CSV generated!!


<h1>3. Scraping Google Maps (perpustakaan universitas brawijaya) reviews</h1>

In [20]:
# parameters passed for API call (scraping all the map data of that place)
# api_key - it is the unique key which we get from serpapi
# engine - SerpApi search engine (google maps, google maps reviews...there are multiple options to choose)
# q - query (here, our map location/place)
# ll - longitude & latitude (GPS coordinates)
# type - here we expect place result. so place option choosed.
# place_id - unique ID of the place (you can get place_id by using google maps API)
# hl - language choosen (here id is Indonesian)
# start - useful for pagination in upcoming steps (to gather all the reviews)
params = {
  'api_key': '<serp api key>',
  'engine': 'google_maps',
  'q': 'perpustakaan Ub',
  'll': '@-7.9531284,112.6108036,17z',
  'type': 'place',
  'place_id':'ChIJB8AHBXqCeC4RciswUQrMiEs',
  'hl': 'id',
  'start': 0,  
}

# make API call, fetch results into dictionary format
search = GoogleSearch(params)
results = search.get_dict()

# parameters passed for API call (scraping only the data related to reviews)
# engine - engine here will be google_maps_reviews
# sort_by - basically sorting most relevant reviews
# data_id - ID of the data (can be obtained through google maps API call or from the place URL.)
params = {
        'api_key': 'd0141422a1be40c488c0f7d7a4b4643702a0443f93ad2a690e6120389c3964b1',
        'engine': 'google_maps_reviews',
        'sort_by': 'qualityScore',
        'hl': 'id',
        'data_id': '0x2e78827a0507c007:0x4b88cc0a51302b72'
    }

# make API call for reviews related data
search = GoogleSearch(params)

In [22]:
# code snippet for pagination (getting all the reviews)

# empty list to store reviews data
reviews = []

# there is a condition inside the loop. If the next_page_token is present, then the search object is updated. Else, the loop is stopped
while True:
    new_reviews_page_result = search.get_dict()
    reviews.extend(new_reviews_page_result['reviews'])
    
    if new_reviews_page_result.get('serpapi_pagination').get('next') and new_reviews_page_result.get('serpapi_pagination').get('next_page_token'):
        search.params_dict.update(dict(parse_qsl(urlsplit(new_reviews_page_result.get('serpapi_pagination', {}).get('next')).query)))
    else:
        break

<h3>Extracting the reviewers names and their reviews, modifying the dataframe to have only Indonesian reviews</h3>

In [24]:
# empty dict to store names and reviews
reviews_dict={}

# empty lists to store names and reviews separately
name_lst, review_lst=[],[]

for i in reviews:
  # storing names and reviews in the respective lists
  name_lst.append(i['user']['name'])
  review_lst.append(i['snippet'])

# storing the names and reviews list into the dictionary, so that it can be converted into a dataframe
reviews_dict['name'] = name_lst
reviews_dict['review'] = review_lst

# create a dataframe
df = pd.DataFrame(reviews_dict)
df

,name,review
0,Fauziah Rachmawati,"Tempatnya luas, lengkap dengan tempat sholat....."
1,Hakim Akbar,"Lokasi yang nyaman untuk belajar, mencari refe..."
2,Annisa Cahya,Sangat nyaman tempatnya di lantai 1 untuk meng...
3,Retno Kusumawardani,Tempat belakar sekaligus nongkring yang asik. ...
4,Putu Reditya,Perpustakaan Universitas Brawijaya terletak di...
...,...,...
391,Shela Hajjaria Putri,
392,intanpermata hadi,
393,Vierkury Metyopandi,
394,Ryandra Ramadhana,


**Observation -** If observed carefully, the API extracted the original Indonesian reviews alongwith the English reviews (here the English reviews has its Indonesian translation as well). We don't need the actual English reviews. <br>
As we sorted the data before, we can locate where the Indonesian reviews finishes. It finishes at 134th index. So we'll just consider rows till 134 index (including 134th index).

In [25]:
# first 135 rows
modified_df = df[:135]
modified_df

,name,review
0,Fauziah Rachmawati,"Tempatnya luas, lengkap dengan tempat sholat....."
1,Hakim Akbar,"Lokasi yang nyaman untuk belajar, mencari refe..."
2,Annisa Cahya,Sangat nyaman tempatnya di lantai 1 untuk meng...
3,Retno Kusumawardani,Tempat belakar sekaligus nongkring yang asik. ...
4,Putu Reditya,Perpustakaan Universitas Brawijaya terletak di...
...,...,...
130,Alfaafan Yusufa Achmad,Mantap
131,Deny Kuswanto (デニ),Nyaman
132,GEFA TV,Menyenangkan..
133,Mochamad Umar Al Hafidz,Perpustakaan UB


<h3>Generating output csv file for indonesian reviews (perpustakaan universitas brawijaya)</h3>

In [26]:
modified_df.to_csv('google_maps_indo_reviews_perpustakaan_universitas_brawijaya.csv')
print('CSV generated!!')

CSV generated!!


<h2>Combining all the 3 csv files into a single csv file</h2>

We got Indonesian reviews for all the 3 keywords provided. These reviews are stored in 3 respective csv files. Time to combine all the 3 csv reviews!!

In [29]:
# merging three csv files
final_df = pd.concat(
	map(pd.read_csv, ['google_maps_indo_reviews.csv', 'google_maps_indo_reviews_perpustakaan_ugm.csv', 'google_maps_indo_reviews_perpustakaan_universitas_brawijaya.csv']), ignore_index=True)

# generating a column named 'Unnamed:0' which is not required. So dropping that column from the dataframe
final_df = final_df.drop(['Unnamed: 0'], axis=1)
final_df

,name,review
0,Dwi Oktarina,Perpustakaan UI bebas diakses sivitas akademik...
1,Ramdanih,Lokasi yang strategis di kawasan Universitas I...
2,Putri Nurfitriani,Nyaman untuk ngerjain tugas buat yg males di k...
3,Kevin Ewaldo,Perpustakaan UI sudah sangat baik. Kalau bicar...
4,Emad Eus,"Perpustakaan yg baik, namun maintenance infras..."
...,...,...
682,Alfaafan Yusufa Achmad,Mantap
683,Deny Kuswanto (デニ),Nyaman
684,GEFA TV,Menyenangkan..
685,Mochamad Umar Al Hafidz,Perpustakaan UB


In [30]:
# generating final csv file which contains all the Indonesian reviews for 3 keywords

final_df.to_csv('google_maps_indonesian_reviews_final.csv')
print('CSV generated!!')

CSV generated!!
